In [5]:
import os
import docx
import re
import pandas as pd 
import numpy as np
import pickle

import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn import metrics
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer as snow



In [3]:
# Загрузка модели 
def get_model(path_model):
    with open(path_model, 'rb') as file:
        pickle_model = pickle.load(file)
    return pickle_model


In [6]:
# Поставить местоположение у тебя
path_model = r"C:\Users\79219\Desktop\digital_xakaton\model\LinearSVC_model.pkl"
model = get_model(path_model)

In [7]:
def load_file(path_to_file):
    doc = docx.Document(path_to_file)
    all_paragraphs = doc.paragraphs
    list_text_doc = [par.text for par in all_paragraphs]
    len_doc_paragraph = len(all_paragraphs)
    return list_text_doc, len_doc_paragraph

In [9]:
path_document = r"C:\Users\79219\Desktop\digital_xakaton\document\Postanovlenie_Pravitelstva_Rossiyskoy_Federatsii_ot_07_04_2022_612.docx"
list_text_doc, len_paragraph = load_file(path_document)

In [10]:
def pandas_df_text(text_doc):
    df = pd.DataFrame(text_doc, columns=['text'])
    return df

In [11]:
df = pandas_df_text(list_text_doc)